In [1]:
val path = "/tool/hadoop_practice/data/lfw/*"
val rdd = sc.wholeTextFiles(path)
val first = rdd.first
println(first)

(file:/tool/hadoop_practice/data/lfw/Alexander_Rumyantsev/Alexander_Rumyantsev_0002.jpg,���� JFIF      �� C 


��  � �" ��           	
�� �   } !1AQa"q2���#B��R��$3br�	
%&'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz���������������������������������������������������������������������������        	
�� �  w !1AQaq"2�B����	#3R�br�
$4�%�&'()*56789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz��������������������������������������������������������������������������   ? ��*<T�P�����҅Ħ[�pu t9x<���Q8��˾%3dI��U8-� �|K�û�7'Սs�FZa[�(�\�� x�+�YFkۣ���yVj MV���F2B
�<��j}v�"��st��=�Z-������#�CsVY�^j"�0N*S"[��$�S@�4���r(W)J�c�:R
GS`��*�H�T0G�֧��a�:�5=�<�Ұ�X�%d�z�6��GP��B ��E�ն%�!#��2Hd95w>r�0���aj
yoĶ#J���^������C6�)�MvS.'�>%?�?����b\��
��"g]������ �{��|fłb ��ۘ
?=*;Y�=�

In [3]:
val files = rdd.map { case (fileName, content) => fileName.replace("file:", "") }
println(files.first)
// file:/PATH/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg
println(files.count)

/tool/hadoop_practice/data/lfw/Alexander_Rumyantsev/Alexander_Rumyantsev_0002.jpg
1059                                                                            


In [4]:
// load an image from a file
import java.awt.image.BufferedImage
def loadImageFromFile(path: String): BufferedImage = {
 import javax.imageio.ImageIO
 import java.io.File
 ImageIO.read(new File(path))
}

In [5]:
val aePath = "/tool/hadoop_practice/data/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg"
val aeImage = loadImageFromFile(aePath)

In [6]:
// convert an image to grayscale, and scale it to new width and height
def processImage(image: BufferedImage, width: Int, height: Int): BufferedImage = {
 val bwImage = new BufferedImage(width, height, BufferedImage.TYPE_BYTE_GRAY)
 val g = bwImage.getGraphics()
 g.drawImage(image, 0, 0, width, height, null)
 g.dispose()
 bwImage
}

In [7]:
val grayImage = processImage(aeImage, 100, 100)

In [8]:
import javax.imageio.ImageIO
import java.io.File
ImageIO.write(grayImage, "jpg", new File("/tmp/aeGray.jpg"))

true

In [9]:
// extract the raw pixels from the image as a Double array
def getPixelsFromImage(image: BufferedImage): Array[Double] = {
 val width = image.getWidth
 val height = image.getHeight
 val pixels = Array.ofDim[Double](width * height)
 image.getData.getPixels(0, 0, width, height, pixels)
 // pixels.map(p => p / 255.0)   // optionally scale to [0, 1] domain
}

// put all the functions together
def extractPixels(path: String, width: Int, height: Int): Array[Double] = {
 val raw = loadImageFromFile(path)
 val processed = processImage(raw, width, height)
 getPixelsFromImage(processed)
}

In [10]:
val pixels = files.map(f => extractPixels(f, 50, 50))
println(pixels.take(10).map(_.take(10).mkString("", ",", ", ...")).mkString("\n"))

178.0,177.0,176.0,172.0,170.0,164.0,90.0,47.0,45.0,43.0, ...
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ...
104.0,84.0,91.0,115.0,123.0,116.0,106.0,96.0,95.0,113.0, ...
22.0,22.0,20.0,20.0,20.0,20.0,21.0,20.0,21.0,21.0, ...
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ...
43.0,48.0,46.0,47.0,48.0,47.0,44.0,44.0,46.0,46.0, ...
228.0,222.0,224.0,232.0,204.0,206.0,212.0,225.0,149.0,188.0, ...
62.0,91.0,90.0,86.0,87.0,88.0,90.0,92.0,94.0,94.0, ...
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ...
198.0,191.0,194.0,192.0,112.0,46.0,42.0,46.0,62.0,71.0, ...


In [11]:
// create vectors
import org.apache.spark.mllib.linalg.Vectors
val vectors = pixels.map(p => Vectors.dense(p))
// the setName method createa a human-readable name that is displayed in the Spark Web UI
vectors.setName("image-vectors")
// remember to cache the vectors to speed up computation
vectors.cache

image-vectors MapPartitionsRDD[4] at map at <console>:47

In [15]:
// normalize the vectors by subtracting the column means
import org.apache.spark.mllib.feature.StandardScaler
val scaler = new StandardScaler(withMean = true, withStd = false).fit(vectors)
val scaledVectors = vectors.map(v => scaler.transform(v))
// create distributed RowMatrix from vectors, and train PCA on it
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.distributed.RowMatrix
val matrix = new RowMatrix(scaledVectors)
val K = 10
val pc = matrix.computePrincipalComponents(K)


Name: java.lang.OutOfMemoryError
Message: Java heap space
StackTrace:   at breeze.linalg.svd$.breeze$linalg$svd$$doSVD_Double(svd.scala:93)
  at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:40)
  at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:39)
  at breeze.generic.UFunc$class.apply(UFunc.scala:48)
  at breeze.linalg.svd$.apply(svd.scala:23)
  at org.apache.spark.mllib.linalg.distributed.RowMatrix.computePrincipalComponentsAndExplainedVariance(RowMatrix.scala:389)
  at org.apache.spark.mllib.linalg.distributed.RowMatrix.computePrincipalComponents(RowMatrix.scala:411)

In [13]:
// use Breeze to save the principal components as a CSV file
val rows = pc.numRows
val cols = pc.numCols
println(rows, cols)

Name: Unknown Error
Message: <console>:33: error: not found: value pc
       val rows = pc.numRows
                  ^
StackTrace: 

In [14]:
import breeze.linalg.DenseMatrix
val pcBreeze = new DenseMatrix(rows, cols, pc.toArray)
import breeze.linalg.csvwrite
import java.io.File
csvwrite(new File("/tmp/pc.csv"), pcBreeze)


Name: Unknown Error
Message: <console>:33: error: not found: value rows
       val pcBreeze = new DenseMatrix(rows, cols, pc.toArray)
                                      ^
<console>:33: error: not found: value cols
       val pcBreeze = new DenseMatrix(rows, cols, pc.toArray)
                                            ^
<console>:33: error: not found: value pc
       val pcBreeze = new DenseMatrix(rows, cols, pc.toArray)
                                                  ^
StackTrace: 